In [3]:
import FinanceDataReader as fdr
import matplotlib.pyplot as plt
%matplotlib inline
import os
import FinanceDataReader as fdr 
import pandas as pd
import numpy as np
import requests
import bs4

pd.options.display.float_format = '{:,.3f}'.format


### FinanceDataReader 로 일봉 데이터 가져오기
가설 분석과 수익율 예측 모델링은 변동성이 큰 코스닥 종목만을 대상으로 하겠습니다. 
가설검정을 위하여 과거 수 개월치의 일봉데이터가 필요합니다. 우선 데이터를 종목별로 가져오기 위해서 FinanceDataReader 의 Stocklisting 메소드에서 코스닥의 종목 코드와 정보를 불러옵니다.

In [ ]:
kosdaq_df = fdr.StockListing('KOSDAQ')
kosdaq_df.head()

<br> 섹터가 정의되지 않은 종목과 2021년 1월 1일 이후 상장된 종목은 제외하겠습니다. 종 1422 개의 종목이 있습니다. 독자분이 책을 보시는 시점에는 종목 수가 바뀌어 있을 것입니다.    
kosdaq_df 에서 필요한 컬럼 'Symbol' 과 'Name' 두 개만 kosdaq_list 에 저장합니다. 그리고 종목코드 'Symbol' 과 'Name' 을 각 각 'code' 외 'name' 으로 바꿔줍니다. 그리고 나중을 위해서 결과물을 pickle 파일로 저장도 합니다.

In [8]:
print(kosdaq_df['Symbol'].nunique())

c1 = (kosdaq_df['ListingDate']>'2021-01-01')
c2 = (kosdaq_df['Sector'].isnull())
print(kosdaq_df[~c1 & ~c2]['Symbol'].nunique())

kosdaq_list = kosdaq_df[~c1 & ~c2][['Symbol','Name','Sector']].rename(columns={'Symbol':'code','Name':'name','Sector':'sector'})
kosdaq_list.to_pickle('kosdaq_list.pkl')

1571
1421


<br> 저장한 pickle 파일을 읽고, sector 가 몇개나 있는 지 세어봅니다. 

In [9]:
kosdaq_list = pd.read_pickle('kosdaq_list.pkl')
kosdaq_list['sector'].nunique()

132

<br> For Loop 에서 kosdaq_list 의 종목을 하나씩 불러서 DataReader 로 2021년 1월 3일부터 2022년 3월 31일까지 일봉데이터를 수집합니다. 

In [57]:
price_data = pd.DataFrame()

for code, name in zip(kosdaq_list['code'], kosdaq_list['name']):  # 코스닥 모든 종목에서 대하여 반복
    daily_price = fdr.DataReader(code,  start='2021-01-03', end='2022-03-31') # 종목, 일봉, 데이터 갯수
    daily_price['code'] = code
    daily_price['name'] = name
    price_data = pd.concat([price_data, daily_price], axis=0)   

price_data.index.name = 'date'
price_data.columns= price_data.columns.str.lower() # 컬럼 이름 소문자로 변경
price_data.to_pickle('stock_data_from_fdr.pkl')

<br> 저장한 pickle 파일을 다시 읽어 첫 5 라인을 head 메소드로 찍어보면 아래와 같습니다. 여기서 date가 인덱스로 처리되어 있다는 것을 기억해주시면 좋습니다. 타이핑 편의를 위해 컬럼이름을 소문자료 변경하겠습니다.  

In [11]:
price_data = pd.read_pickle('stock_data_from_fdr.pkl')
price_data.head()

,open,high,low,close,volume,change,code,name
date,,,,,,,,
2021-01-04,2185,2320,2135,2260,588133,0.044,060310,3S
2021-01-05,2270,2285,2200,2250,410263,-0.004,060310,3S
2021-01-06,2225,2310,2215,2290,570349,0.018,060310,3S
2021-01-07,2290,2340,2240,2290,519777,0.000,060310,3S
2021-01-08,2300,2315,2225,2245,462568,-0.020,060310,3S


<br> 몇 개의 종목이 있고, 각 종목별 일봉의 갯 수 가 몇 개인지 확인해 보겠습니다. 종목 수는 1422 개, 307 개의 일봉이 있습니다.

In [12]:
print(price_data['code'].nunique())
print(price_data.groupby('code')['close'].count().agg(['min','max']))

1422
min    307
max    307
Name: close, dtype: int64


<br>

### 네이버 증권 웹크롤링으로 일봉 가져오기
이 번에는 네이버 증권 차트 _(네이버 차트 예시 필요)_ 에서 데이터를 가져오는 방법도 시도해 보겠습니다. 웹 크롤링은 코드가 복잡합니다. 첫 번째 방법인 FinanceDataReader 로 추출하는 방법을 추천드립니다.   
다시 pickle 파일을 읽습니다. make_price_data 함수는 '종목', '추출단위', '데이터 건수' 를 인자로 네이버증권에서 데이터를 가져오는 함수입니다. 인자는 작은 따옴표에 넣어야 합니다. 셀트리온 헬스케어(091990) 의 일봉 데이터를 최근 300 일 가져오고 싶다면  make_price_data('091990', 'day', '300') 와 같이 호출합니다. 이 함수를 for 문을 이용해 모든 코스닥 종목에서 대하여 호출하고, 각 결과를 price_data 라는 데이터프레임에 담습니다. 
for 문을 돌리고 결과를 concat 함수로 연속으로 저장하는 방법은 자주 활용되는 기법입니다. 

In [46]:
# 네이버 증권 차트에서 데이터 크롤링

kosdaq_list = pd.read_pickle('kosdaq_list.pkl')

def make_price_data(code, name, timeframe, count):
    url = 'https://fchart.stock.naver.com/sise.nhn?symbol=' + code + '&timeframe=' + timeframe + '&count=' + count + '&requestType=0'
    price_data = requests.get(url)
    price_data_bs = bs4.BeautifulSoup(price_data.text, 'lxml')
    item_list = price_data_bs.find_all('item')

    date_list = [] 
    open_list = []
    high_list = []
    low_list = []
    close_list = []
    trade_list = []

    for item in item_list:
        data = item['data'].split('|')
        date_list.append(data[0])
        open_list.append(data[1])
        high_list.append(data[2])
        low_list.append(data[3])
        close_list.append(data[4])
        trade_list.append(data[5])        

    price_df = pd.DataFrame({'open': open_list, 'high': high_list, 'low': low_list, 'close': close_list, 'volume': trade_list}, index=date_list)            
    price_df['code'] = code
    price_df['name'] = name
    num_vars = ['open','high','low','close','volume']
    char_vars = ['code','name']
    price_df = price_df.reindex(columns = char_vars + num_vars)

    for var in num_vars:
        price_df[var] = pd.to_numeric(price_df[var], errors='coerce')

    price_df.index = pd.to_datetime(price_df.index, errors='coerce')

    return price_df

price_data = pd.DataFrame()

for code, name in zip(kosdaq_list['code'], kosdaq_list['name']):  # 코스닥 모든 종목에서 대하여 반복
    daily_price = make_price_data(code, name, 'day', '307') # 종목, 일봉, 데이터 갯수
    price_data = pd.concat([price_data, daily_price], axis=0)   

price_data.index.name = 'date'
price_data.to_pickle('stock_data_from_naver.pkl')

<br> 저장한 pickle 파일을 다시 읽어 첫 5 라인을 head 메소드로 찍어보면 아래와 같습니다. 여기서 date가 인덱스로 처리되어 있다는 것을 기억해주시면 좋습니다.  

In [47]:
price_data = pd.read_pickle('stock_data_from_naver.pkl')
price_data.head()

,code,name,open,high,low,close,volume
date,,,,,,,
2021-02-08,060310,3S,2650,2655,2565,2610,663393
2021-02-09,060310,3S,2605,2745,2585,2735,964973
2021-02-10,060310,3S,2740,2765,2625,2730,520334
2021-02-15,060310,3S,2750,2765,2680,2720,488060
2021-02-16,060310,3S,2720,2745,2645,2715,453009


<br> 몇 개의 종목이 있고, 각 종목별 일봉의 갯 수 가 몇 개인지 확인해 보겠습니다. 종목 수는 1422 개, 307 개의 일봉이 있습니다.

In [50]:
print(price_data['code'].nunique())
print(price_data.groupby('code')['close'].count().agg(['min','max']))

1422
min    307
max    307
Name: close, dtype: int64


In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\shinki\\OneDrive - Novelis Inc\\Documents\\Data Science\\10. Learning\\22. Hanbit Media'